# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [1]:
import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [2]:
# weight: num_embeddings x embedding_dim
e_layer = nn.Embedding(
    num_embeddings=10,    # embedding할 총 단어수(vocab_size, n_vocab)
    embedding_dim=5,        # embedding vector의 차원수. (한개 단어를 몇개 숫자 vector로 표현)
)

In [5]:
# weight 조회
print(e_layer.weight.shape)
e_layer.weight

torch.Size([10, 5])


Parameter containing:
tensor([[ 1.2872, -0.9666,  1.0572,  1.8553,  1.4034],
        [-0.2536,  0.4192, -2.0518,  0.0535, -1.4707],
        [ 0.3092, -0.5023, -1.0968, -1.5061,  0.2261],
        [-0.7654, -2.6820,  1.2210, -0.9326, -0.6549],
        [ 1.0581,  0.7595,  0.0052,  0.8301, -1.6413],
        [ 0.3635,  0.5969, -0.1236, -1.2737, -0.3104],
        [-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],
        [ 0.0207, -0.5517,  0.8156, -0.9954,  2.0784],
        [ 0.3543,  0.1219, -1.8038,  0.9530,  0.1627],
        [ 0.9938,  1.4051,  0.1349,  0.3557,  1.7975]], requires_grad=True)

In [6]:
# 단어 index 는 정수여야 한다. 
input_data = torch.LongTensor([[3, 6, 1]])    # dtype=int64
result = e_layer(input_data)
result.shape   # [1:문서개수,    3:문서의 토큰수,    5:embedding차원수]

torch.Size([1, 3, 5])

In [8]:
e_layer.weight

Parameter containing:
tensor([[ 1.2872, -0.9666,  1.0572,  1.8553,  1.4034],
        [-0.2536,  0.4192, -2.0518,  0.0535, -1.4707],
        [ 0.3092, -0.5023, -1.0968, -1.5061,  0.2261],
        [-0.7654, -2.6820,  1.2210, -0.9326, -0.6549],
        [ 1.0581,  0.7595,  0.0052,  0.8301, -1.6413],
        [ 0.3635,  0.5969, -0.1236, -1.2737, -0.3104],
        [-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],
        [ 0.0207, -0.5517,  0.8156, -0.9954,  2.0784],
        [ 0.3543,  0.1219, -1.8038,  0.9530,  0.1627],
        [ 0.9938,  1.4051,  0.1349,  0.3557,  1.7975]], requires_grad=True)

In [12]:
result[0, 1]


tensor([-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],
       grad_fn=<SelectBackward0>)

In [ ]:
[-1.4983,  1.6032,  0.9305,  1.3301,  0.0593],

# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

In [13]:
!pip install korpora

  Using cached Korpora-0.2.0-py3-none-any.whl.metadata (26 kB)
  Using cached dataclasses-0.6-py3-none-any.whl.metadata (3.0 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached Korpora-0.2.0-py3-none-any.whl (57 kB)
Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


## Dataset, DataLoader 생성

### Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://ko-nlp.github.io/Korpora/ko-docs/corpuslist/nsmc.html
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
#### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

In [1]:
from Korpora import Korpora
import pandas as pd

In [3]:
### 데이터 로딩 (다운로딩 + 로딩)
corpus = Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\USER\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\USER

In [4]:
type(corpus)

Korpora.korpus_nsmc.NSMCKorpus

In [5]:
# 전체 다 읽어오기
all_text = corpus.get_all_texts()  # (모든 input)
type(all_text), len(all_text)

(tuple, 200000)

In [6]:
print(all_text[0])

아 더빙.. 진짜 짜증나네요 목소리


In [7]:
all_label = corpus.get_all_labels() # list(모든 label)
print(type(all_label), len(all_label))
print(all_label[0])

<class 'list'> 200000
0


In [8]:
## train/test 분리해서 로딩
tr = corpus.train
len(tr.texts), len(tr.labels)

(150000, 150000)

In [9]:
te = corpus.test
# te.texts[0]
len(te.texts), len(te.labels)

(50000, 50000)

# 토큰화 (Tokenization)
1. cleaning (전처리)
2. subword 방식 토큰화

In [10]:
# pip install    jpype1  konlpy

In [26]:
#########################
# cleaning 전처리 - 함수
#########################
from konlpy.tag import Okt
import re
import string

def text_preprocessing(text, tokenizer):
    """
    1. 영문자를 소문자 변경.
    2. 구두점 제거
    3. 토큰화 -> 형태소기반으로 토큰화. 어간을 추출해서 토큰화.  
    # 소문자변환, 어간으로 토큰화 -> 같은 단어는 같은 표현으로 만든다.
    """
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", ' ', text)
    text = tokenizer.morphs(text, stem=True) # stem=True: 어간 추출
    return " ".join(text)

In [27]:
text = "이 Movie 정말 재미 있어요....,!!!!!!. 꼭보세요.### 저기 있네요."
okt = Okt()
text_preprocessing(text, okt)
# text = text.lower()
# text = re.sub(f"[{string.punctuation}]", ' ', text)
# text = okt.morphs(text, stem=True) # stem=True: 어간 추출
# " ".join(text)  # 리스트의 원소들을 " "을 구분자로 하나의 string으로 합친다.

'이 movie 정말 재미 있다 꼭 보다 저기 있다'

In [ ]:
########################
# 전처리
########################
okt = Okt()
all_text = [text_preprocessing(text, okt)  for text in all_text]

## 모델
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

### 모델 정의

## 학습

### Train/Test 함수 정의

### 모델저장